In [1]:
import numpy as np
from PIL import Image
import os
import shutil
from collections import defaultdict
import random
import json
random.seed(42)


def resize_pad(image, size):
    """
    Resize and pad an image to a square image of size size
    """
    # Resize
    width, height = image.size
    if width > height:
        image = image.resize((size, int(height * size / width)))
    else:
        image = image.resize((int(width * size / height), size))

    #make the image black and white, not grayscale
    image = image.convert('L')
    # Pad
    new_image = Image.new('L', (size, size), 255)
    new_image.paste(image, ((size - image.size[0]) // 2, (size - image.size[1]) // 2))
    return new_image

In [2]:

SRC_DIR = "base"
DST_DIR = "EVAL"

for filename in os.listdir(DST_DIR):
    file_path = os.path.join(DST_DIR, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)

    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [9]:
global_hashes = set()


def make_combinations(folder1, folder2, dst_dir, priority = False):

    if "porsche" in folder1:
        folder1_type = "porsche"
    else:
        folder1_type = "lamborghini"
    
    if "lambo" in folder2:
        folder2_type = "lamborghini"
    else:
        folder2_type = "porsche"

    i = len(os.listdir(dst_dir))
    for filename1 in os.listdir(folder1):
        for filename2 in os.listdir(folder2):
            file_path1 = os.path.join(folder1, filename1)
            file_path2 = os.path.join(folder2, filename2)
            
            if file_path1 == file_path2:
                continue
            
            img1 = Image.open(file_path1)
            img2 = Image.open(file_path2)
            img1 = resize_pad(img1, 256)
            img2 = resize_pad(img2, 256)
            
            new_img = Image.new('L', (512, 256))
            new_img.paste(img1, (0, 0))
            new_img.paste(img2, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)

            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}_p.png"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder1_type}_{folder2_type}.png"))
            i += 1

            new_img = Image.new('L', (512, 256))
            new_img.paste(img2, (0, 0))
            new_img.paste(img1, (256, 0))

            h = hash(new_img.tobytes())
            if h in global_hashes:
                continue
            global_hashes.add(h)
            
            if priority:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}_p.png"))
            else:
                new_img.save(os.path.join(dst_dir, f"{i:04d}_{folder2_type}_{folder1_type}.png"))
            i += 1


In [10]:
make_combinations(f"base/lambo", f"base/porsche", DST_DIR)

In [11]:
make_combinations(f"base/lambo", f"base/lambo", DST_DIR)
make_combinations(f"base/porsche", f"base/porsche", DST_DIR)

In [13]:
EVAL_DIR = "EVAL"

lambo_lambo = []
porsche_porsche = []
lambo_porsche = []
porsche_lambo = []
for file in os.listdir(EVAL_DIR):
    if "lamborghini_lamborghini" in file:
        lambo_lambo.append(file)
    elif "porsche_porsche" in file:
        porsche_porsche.append(file)
    elif "lamborghini_porsche" in file:
        lambo_porsche.append(file)
    elif "porsche_lamborghini" in file:
        porsche_lambo.append(file)

random.shuffle(lambo_lambo)
random.shuffle(porsche_porsche)
random.shuffle(lambo_porsche)
random.shuffle(porsche_lambo)

#select 50 and remove the rest
lambo_lambo = lambo_lambo[:50]
porsche_porsche = porsche_porsche[:50]
lambo_porsche = lambo_porsche[:50]
porsche_lambo = porsche_lambo[:50]

for file in os.listdir(EVAL_DIR):
    if file not in lambo_lambo and file not in porsche_porsche and file not in lambo_porsche and file not in porsche_lambo:
        os.remove(os.path.join(EVAL_DIR, file))
        

### CREATE JSON

In [14]:
import json

for_json = []

template1 = {
    "question": "Which car is a Porsche?",
    "options": [
      "The left car",
      "The right car",
      "Both cars",
      "Neither car"
    ],
    "answer": None,
    "image": None,
    "category": "ordinary_soft_localization"
  }


template2 = {
    "question": "Which car is a Lamborghini?",
    "options": [
      "The left car",
      "The right car",
      "Both cars",
      "Neither car"
    ],
    "answer": None,
    "image": None,
    "category": "ordinary_soft_localization"
  }


for file in os.listdir(EVAL_DIR):
    if "lamborghini_lamborghini" in file:
        template1["image"] = file
        template1["answer"] = "Neither car"
        for_json.append(template1.copy())

        template2["image"] = file
        template2["answer"] = "Both cars"
        for_json.append(template2.copy())

    elif "porsche_porsche" in file:
        template1["image"] = file
        template1["answer"] = "Both cars"
        for_json.append(template1.copy())

        template2["image"] = file
        template2["answer"] = "Neither car"
        for_json.append(template2.copy())

    elif "lamborghini_porsche" in file:
        template1["image"] = file
        template1["answer"] = "The right car"
        for_json.append(template1.copy())

        template2["image"] = file
        template2["answer"] = "The left car"
        for_json.append(template2.copy())

    elif "porsche_lamborghini" in file:
        template1["image"] = file
        template1["answer"] = "The left car"
        for_json.append(template1.copy())

        template2["image"] = file
        template2["answer"] = "The right car"
        for_json.append(template2.copy())


In [16]:
with open("eval_labels.json", "w") as f:
    json.dump(for_json, f, indent=4)